In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import norm
from sklearn.model_selection import GridSearchCV

In [2]:


def calculate_probability_of_sale(row):
    price_difference = row['sugestao_preco'] - row['precoconcorrente']
    std_deviation = row['preco_real'] - row['sugestao_preco']
    if std_deviation == 0:
        return 'N/A'
    z_score = price_difference / std_deviation
    probability = 1 - norm.cdf(z_score)  
    return '{:.2%}'.format(probability)

def train_and_predict(csv_file_path):
    dados = pd.read_csv(csv_file_path, sep=";", encoding="latin-1")
    dados.fillna(0, inplace=True)
    dados[['preco', 'margem', 'precoconcorrente']] = dados[['preco', 'margem', 'precoconcorrente']].applymap(
        lambda k: float(str(k).replace(",", "").replace(".", "")))

    object_columns = dados.select_dtypes(include=['object']).columns
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded_data = encoder.fit_transform(dados[object_columns])
    encoded_columns = encoder.get_feature_names_out(object_columns)
    encoded_df = pd.DataFrame(encoded_data, columns=encoded_columns)

    dados_encoded = pd.concat([dados.drop(object_columns, axis=1), encoded_df], axis=1)

    X = dados_encoded.drop("preco", axis=1)
    y = dados["preco"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    param_grid = {
        'n_estimators': [100, 500, 1000],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }

    regressor = XGBRegressor(random_state=42)
    grid_search = GridSearchCV(regressor, param_grid, cv=3, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_regressor = grid_search.best_estimator_

    
    results_df = pd.DataFrame({
        'nomeproduto': dados.loc[X_test.index, 'nomeproduto'], 
        'precoconcorrente': X_test['precoconcorrente'],
        'preco_real': y_test,
        'sugestao_preco': best_regressor.predict(X_test)
    })

   
    results_df['diferenca_preco'] = results_df['sugestao_preco'] - results_df['precoconcorrente']

    
    results_df['probabilidade_venda'] = results_df.apply(calculate_probability_of_sale, axis=1)

    
    pd.set_option('display.float_format', '{:.2f}'.format)

    return results_df

csv_file_path = r'D:\dev0608\google_teste.csv'
results_dataframe = train_and_predict(csv_file_path)

print(results_dataframe)


                                           nomeproduto  precoconcorrente  \
237                Registro de Gaveta ABNT 1.1/4 Docol          10699.00   
116                        Lavatório Aspen Branco Deca          21445.00   
113                        Lavatório Aspen Branco Deca        2136845.00   
42   Acabamento Para Monocomando 4 Vias Drop Cromad...           4809.00   
126                Registro de Gaveta ABNT 1.1/4 Docol        9487045.00   
..                                                 ...               ...   
173             Cuba de Sobrepor Smart 100x50cm Franke         148299.00   
72   Acabamento Para Monocomando 4 Vias Drop Cromad...          82271.00   
75   Acabamento Para Monocomando 4 Vias Drop Cromad...          72454.00   
16   Acabamento Para Monocomando 4 Vias Drop Cromad...          13699.00   
66                         Lavatório Aspen Branco Deca          68022.00   

     preco_real  sugestao_preco  diferenca_preco probabilidade_venda  
237   111856.00 